In [2]:
import pandas as pd

In [65]:
df = pd.read_csv("spectra_raw_all - Sheet1(2).csv")
df

,Wavelength,B_B,B_B.1,B_B.2,C7_buffer,C7_buffer.1,C7_buffer.2,B_PC1,B_PC1.1,B_PC1.2,...,B_PB50.2,C7_PB50_1,C7_PB50_1.1,C7_PB50_1.2,C7_PB50_2,C7_PB50_2.1,C7_PB50_2.2,C7_PB50_3,C7_PB50_3.1,C7_PB50_3.2
0,Wavelength_200,2.646,2.643,2.650,2.676,2.678,2.678,2.699,2.700,2.697,...,2.657,2.664,2.669,2.669,2.668,2.675,2.670,2.677,2.669,2.672
1,Wavelength_201,2.614,2.616,2.618,2.643,2.642,2.645,2.658,2.654,2.655,...,2.625,2.627,2.622,2.631,2.631,2.630,2.626,2.632,2.632,2.632
2,Wavelength_202,2.644,2.646,2.641,2.675,2.672,2.675,2.683,2.681,2.680,...,2.657,2.662,2.657,2.657,2.655,2.663,2.657,2.656,2.657,2.653
3,Wavelength_203,2.694,2.696,2.698,2.728,2.738,2.734,2.736,2.737,2.741,...,2.720,2.712,2.706,2.715,2.713,2.716,2.713,2.711,2.717,2.712
4,Wavelength_204,2.741,2.737,2.738,2.776,2.781,2.775,2.782,2.788,2.782,...,2.763,2.751,2.753,2.752,2.751,2.751,2.753,2.756,2.762,2.756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Wavelength_896,0.041,0.040,0.042,0.041,0.042,0.041,0.041,0.041,0.043,...,0.046,0.045,0.045,0.045,0.045,0.045,0.046,0.045,0.046,0.046
697,Wavelength_897,0.041,0.040,0.042,0.041,0.042,0.041,0.041,0.041,0.043,...,0.046,0.045,0.046,0.045,0.045,0.046,0.046,0.045,0.046,0.046
698,Wavelength_898,0.041,0.040,0.042,0.041,0.043,0.041,0.042,0.041,0.043,...,0.046,0.045,0.046,0.045,0.044,0.046,0.046,0.045,0.046,0.046
699,Wavelength_899,0.041,0.040,0.042,0.041,0.043,0.041,0.041,0.041,0.043,...,0.046,0.045,0.045,0.045,0.044,0.045,0.045,0.045,0.046,0.046


In [66]:
import pandas as pd
import numpy as np

# total NaNs per column
nan_count = df.isna().sum()

print("Total NaN summary:")
print(nan_count[nan_count > 0].sort_values(ascending=False))
print(f"\nTotal columns with at least one NaN: {sum(nan_count > 0)} / {df.shape[1]}")


Total NaN summary:
Series([], dtype: int64)

Total columns with at least one NaN: 0 / 2094


In [67]:
non_numeric_cols = []

for c in df.columns:
    if c == "WAVELENGTH":
        continue
    # try converting to numeric; if any fail → text present
    try:
        pd.to_numeric(df[c], errors='raise')
    except Exception:
        non_numeric_cols.append(c)

print(f"\n Columns containing text / mixed data: {len(non_numeric_cols)}")
if non_numeric_cols:
    print(non_numeric_cols[:15])  # show first few



 Columns containing text / mixed data: 1
['Wavelength']


In [68]:
def average_replicate_columns(df):
    """
    Groups columns by their prefix (before the first '.') and calculates the mean.
    
    The 'Wavelength' column is kept as is.
    
    Args:
        df (pd.DataFrame): The input DataFrame.
        
    Returns:
        pd.DataFrame: A new DataFrame with averaged columns.
    """
    # Keep the 'Wavelength' column separate if it exists
    if 'Wavelength' in df.columns:
        wavelength_col = df['Wavelength']
        df_data = df.drop(columns=['Wavelength'])
    else:
        wavelength_col = None
        df_data = df.copy()

    # Determine the base name for each column
    # For 'B_B', 'B_B.1', 'B_B.2', the base name is 'B_B'
    # For 'C7_buffer', 'C7_buffer.1', 'C7_buffer.2', the base name is 'C7_buffer'
    column_mapping = {col: col.split('.')[0] for col in df_data.columns}
    
    # Create a Series to group columns by their base name
    column_groups = pd.Series(column_mapping)
    print(column_groups)
    
    # Use groupby on the columns axis (axis=1) and calculate the mean
    # This automatically averages 'B_B', 'B_B.1', 'B_B.2' into a single 'B_B' column, etc.
    averaged_df = df_data.groupby(column_groups, axis=1).mean()

    # Re-insert the 'Wavelength' column if it was present
    if wavelength_col is not None:
        averaged_df.insert(0, 'Wavelength', wavelength_col)
        
    return averaged_df

# Example usage (assuming your DataFrame is named 'data_df')
averaged_data = average_replicate_columns(df)
print(averaged_data.head())

B_B                  B_B
B_B.1                B_B
B_B.2                B_B
C7_buffer      C7_buffer
C7_buffer.1    C7_buffer
                 ...    
C7_PB50_2.1    C7_PB50_2
C7_PB50_2.2    C7_PB50_2
C7_PB50_3      C7_PB50_3
C7_PB50_3.1    C7_PB50_3
C7_PB50_3.2    C7_PB50_3
Length: 2093, dtype: object
       Wavelength       B_B     B_PB1    B_PB10    B_PB11    B_PB12    B_PB13  \
0  Wavelength_200  2.646333  2.653000  2.664000  2.665000  2.660333  2.668667   
1  Wavelength_201  2.616000  2.622667  2.626333  2.625000  2.627000  2.626667   
2  Wavelength_202  2.643667  2.645000  2.656333  2.646667  2.650333  2.659000   
3  Wavelength_203  2.696000  2.709333  2.710667  2.708333  2.712667  2.713333   
4  Wavelength_204  2.738667  2.751333  2.758333  2.746667  2.750667  2.759000   

     B_PB14    B_PB15    B_PB16  ...  C7_PN7_1  C7_PN7_2  C7_PN7_3  C7_PN8_1  \
0  2.665667  2.653667  2.648000  ...  2.693000  2.689667  2.697000  2.698333   
1  2.625000  2.623667  2.622333  ...  2.655667  2.

In [69]:
averaged_data

,Wavelength,B_B,B_PB1,B_PB10,B_PB11,B_PB12,B_PB13,B_PB14,B_PB15,B_PB16,...,C7_PN7_1,C7_PN7_2,C7_PN7_3,C7_PN8_1,C7_PN8_2,C7_PN8_3,C7_PN9_1,C7_PN9_2,C7_PN9_3,C7_buffer
0,Wavelength_200,2.646333,2.653000,2.664000,2.665000,2.660333,2.668667,2.665667,2.653667,2.648000,...,2.693000,2.689667,2.697000,2.698333,2.703333,2.703000,2.688667,2.684667,2.691000,2.677333
1,Wavelength_201,2.616000,2.622667,2.626333,2.625000,2.627000,2.626667,2.625000,2.623667,2.622333,...,2.655667,2.650000,2.662667,2.663333,2.667000,2.667333,2.653000,2.655000,2.657333,2.643333
2,Wavelength_202,2.643667,2.645000,2.656333,2.646667,2.650333,2.659000,2.652333,2.652333,2.652667,...,2.683667,2.683000,2.689667,2.687667,2.691333,2.699333,2.680667,2.682333,2.689333,2.674000
3,Wavelength_203,2.696000,2.709333,2.710667,2.708333,2.712667,2.713333,2.714000,2.711333,2.710000,...,2.741000,2.740333,2.749333,2.745000,2.748667,2.749667,2.737667,2.738333,2.737667,2.733333
4,Wavelength_204,2.738667,2.751333,2.758333,2.746667,2.750667,2.759000,2.756333,2.752000,2.752000,...,2.786667,2.782667,2.790333,2.792000,2.792667,2.796000,2.781000,2.776333,2.781667,2.777333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Wavelength_896,0.041000,0.042000,0.044333,0.043333,0.043333,0.048667,0.044000,0.044333,0.043000,...,0.047000,0.047333,0.050000,0.049000,0.048333,0.051000,0.044667,0.045333,0.048333,0.041333
697,Wavelength_897,0.041000,0.041667,0.044333,0.043333,0.043000,0.048000,0.043667,0.044333,0.043000,...,0.047000,0.047667,0.050000,0.049000,0.048333,0.051333,0.045000,0.045667,0.048667,0.041333
698,Wavelength_898,0.041000,0.041667,0.044333,0.043333,0.043000,0.048333,0.044000,0.044667,0.043000,...,0.047333,0.048000,0.050000,0.049000,0.048667,0.051333,0.045333,0.045333,0.048667,0.041667
699,Wavelength_899,0.041000,0.042333,0.044333,0.043333,0.043333,0.048333,0.043667,0.044667,0.043333,...,0.047000,0.047667,0.050000,0.049000,0.048333,0.051000,0.045000,0.045333,0.048333,0.041667


In [70]:
corrections_df = pd.read_csv("Raw plasma data - corrections(1).csv")
corrections_df

,Sample Id,Correction
0,PB1,1.283136
1,PB2,1.020090
2,PB3,0.932184
3,PB4,3.321762
4,PB5,1.037099
...,...,...
169,PC74,0.830737
170,PC75,0.806463
171,PC76,0.686166
172,PC77,1.022668


In [71]:
# Create a simple correction map: {"PB1": value, "PB2": value, ...}
corrections_map = dict(zip(corrections_df["Sample Id"], corrections_df["Correction"]))
corrections_map

{'PB1': 1.283135637,
 'PB2': 1.020090201,
 'PB3': 0.9321843390000001,
 'PB4': 3.32176235,
 'PB5': 1.037098791,
 'PB6': 0.893678161,
 'PB7': 0.925251023,
 'PB8': 0.9876935290000001,
 'PB9': 0.892076013,
 'PB10': 3.133501259,
 'PB11': 0.9287047409999999,
 'PB12': 1.199614272,
 'PB13': 0.925251023,
 'PB14': 0.9287047409999999,
 'PB15': 1.0822096559999999,
 'PB16': 0.7070190390000001,
 'PB17': 1.052453469,
 'PB18': 0.825481088,
 'PB19': 0.892076013,
 'PB20': 0.8794627079999999,
 'PB21': 0.72557597,
 'PB22': 0.906053184,
 'PB23': 1.879172714,
 'PB24': 0.884392077,
 'PB25': 0.780440024,
 'PB26': 0.719705392,
 'PB27': 0.658403255,
 'PB28': 1.008372274,
 'PB29': 1.049230146,
 'PB30': 0.7935948509999999,
 'PB31': 0.7816178690000001,
 'PB32': 0.680015756,
 'PB33': 0.862304362,
 'PB34': 0.760946224,
 'PB35': 0.7875608270000001,
 'PB36': 0.641282813,
 'PB37': 0.728820715,
 'PB38': 0.856599405,
 'PB39': 0.7127718140000001,
 'PB40': 0.87839213,
 'PB41': 0.767714201,
 'PB42': 0.705009529,
 'PB43': 1.

In [72]:
# Create a list to store summary info
correction_summary = []

# Apply corrections
for sample_id, factor in corrections_map.items():
    # Find columns that contain the sample ID
    pattern = rf"(?:^|_)({sample_id})(?:_|$)"
    matching_cols = [col for col in averaged_data.columns if re.search(pattern, col)]

    if matching_cols:
        averaged_data[matching_cols] = averaged_data[matching_cols].apply(lambda col: col * factor)

        # Store which columns were corrected and with what factor
        correction_summary.append({
            "Sample ID": sample_id,
            "Correction Factor": factor,
            "Corrected Columns": matching_cols
        })


In [73]:
for entry in correction_summary:
    print(f"{entry['Sample ID']}: Factor = {entry['Correction Factor']}")
    for col in entry['Corrected Columns']:
        print(f"   - {col}")


PB1: Factor = 1.283135637
   - B_PB1
   - C7_PB1_1
   - C7_PB1_2
   - C7_PB1_3
PB2: Factor = 1.020090201
   - B_PB2
   - C7_PB2_1
   - C7_PB2_2
   - C7_PB2_3
PB3: Factor = 0.9321843390000001
   - B_PB3
   - C7_PB3_1
   - C7_PB3_2
   - C7_PB3_3
PB4: Factor = 3.32176235
   - B_PB4
   - C7_PB4_1
   - C7_PB4_2
   - C7_PB4_3
PB5: Factor = 1.037098791
   - B_PB5
   - C7_PB5_1
   - C7_PB5_2
   - C7_PB5_3
PB6: Factor = 0.893678161
   - B_PB6
   - C7_PB6_1
   - C7_PB6_2
   - C7_PB6_3
PB7: Factor = 0.925251023
   - B_PB7
   - C7_PB7_1
   - C7_PB7_2
   - C7_PB7_3
PB8: Factor = 0.9876935290000001
   - B_PB8
   - C7_PB8_1
   - C7_PB8_2
   - C7_PB8_3
PB9: Factor = 0.892076013
   - B_PB9
   - C7_PB9_1
   - C7_PB9_2
   - C7_PB9_3
PB10: Factor = 3.133501259
   - B_PB10
   - C7_PB10_1
   - C7_PB10_2
   - C7_PB10_3
PB11: Factor = 0.9287047409999999
   - B_PB11
   - C7_PB11_1
   - C7_PB11_2
   - C7_PB11_3
PB12: Factor = 1.199614272
   - B_PB12
   - C7_PB12_1
   - C7_PB12_2
   - C7_PB12_3
PB13: Factor = 0.

In [74]:
averaged_data

,Wavelength,B_B,B_PB1,B_PB10,B_PB11,B_PB12,B_PB13,B_PB14,B_PB15,B_PB16,...,C7_PN7_1,C7_PN7_2,C7_PN7_3,C7_PN8_1,C7_PN8_2,C7_PN8_3,C7_PN9_1,C7_PN9_2,C7_PN9_3,C7_buffer
0,Wavelength_200,2.646333,3.404159,8.347647,2.474998,3.191374,2.469187,2.475617,2.871824,1.872186,...,1.777638,1.775438,1.780278,1.783670,1.786975,1.786755,1.750135,1.747531,1.751653,2.677333
1,Wavelength_201,2.616000,3.365237,8.229619,2.437850,3.151387,2.430326,2.437850,2.839357,1.854040,...,1.752994,1.749254,1.757615,1.760534,1.762958,1.763178,1.726918,1.728220,1.729739,2.643333
2,Wavelength_202,2.643667,3.393894,8.323624,2.457972,3.179378,2.460242,2.463235,2.870381,1.875486,...,1.771477,1.771037,1.775438,1.776619,1.779043,1.784331,1.744927,1.746012,1.750569,2.674000
3,Wavelength_203,2.696000,3.476442,8.493877,2.515242,3.254154,2.510514,2.520505,2.934231,1.916022,...,1.809322,1.808882,1.814823,1.814518,1.816942,1.817603,1.782030,1.782464,1.782030,2.733333
4,Wavelength_204,2.738667,3.530334,8.643241,2.550842,3.299739,2.552768,2.559820,2.978241,1.945716,...,1.839467,1.836826,1.841887,1.845586,1.846027,1.848230,1.810237,1.807199,1.810671,2.777333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Wavelength_896,0.041000,0.053892,0.138919,0.040244,0.051983,0.045029,0.040863,0.047978,0.030402,...,0.031025,0.031245,0.033005,0.032390,0.031950,0.033712,0.029075,0.029509,0.031462,0.041333
697,Wavelength_897,0.041000,0.053464,0.138919,0.040244,0.051583,0.044412,0.040553,0.047978,0.030402,...,0.031025,0.031465,0.033005,0.032390,0.031950,0.033933,0.029292,0.029726,0.031679,0.041333
698,Wavelength_898,0.041000,0.053464,0.138919,0.040244,0.051583,0.044720,0.040863,0.048339,0.030402,...,0.031245,0.031685,0.033005,0.032390,0.032170,0.033933,0.029509,0.029509,0.031679,0.041667
699,Wavelength_899,0.041000,0.054319,0.138919,0.040244,0.051983,0.044720,0.040553,0.048339,0.030637,...,0.031025,0.031465,0.033005,0.032390,0.031950,0.033712,0.029292,0.029509,0.031462,0.041667


In [75]:
corrected_data=averaged_data

In [78]:
corrected_data

,Wavelength,B_B,B_PB1,B_PB10,B_PB11,B_PB12,B_PB13,B_PB14,B_PB15,B_PB16,...,C7_PN7_1,C7_PN7_2,C7_PN7_3,C7_PN8_1,C7_PN8_2,C7_PN8_3,C7_PN9_1,C7_PN9_2,C7_PN9_3,C7_buffer
0,Wavelength_200,2.646333,3.404159,8.347647,2.474998,3.191374,2.469187,2.475617,2.871824,1.872186,...,1.777638,1.775438,1.780278,1.783670,1.786975,1.786755,1.750135,1.747531,1.751653,2.677333
1,Wavelength_201,2.616000,3.365237,8.229619,2.437850,3.151387,2.430326,2.437850,2.839357,1.854040,...,1.752994,1.749254,1.757615,1.760534,1.762958,1.763178,1.726918,1.728220,1.729739,2.643333
2,Wavelength_202,2.643667,3.393894,8.323624,2.457972,3.179378,2.460242,2.463235,2.870381,1.875486,...,1.771477,1.771037,1.775438,1.776619,1.779043,1.784331,1.744927,1.746012,1.750569,2.674000
3,Wavelength_203,2.696000,3.476442,8.493877,2.515242,3.254154,2.510514,2.520505,2.934231,1.916022,...,1.809322,1.808882,1.814823,1.814518,1.816942,1.817603,1.782030,1.782464,1.782030,2.733333
4,Wavelength_204,2.738667,3.530334,8.643241,2.550842,3.299739,2.552768,2.559820,2.978241,1.945716,...,1.839467,1.836826,1.841887,1.845586,1.846027,1.848230,1.810237,1.807199,1.810671,2.777333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Wavelength_896,0.041000,0.053892,0.138919,0.040244,0.051983,0.045029,0.040863,0.047978,0.030402,...,0.031025,0.031245,0.033005,0.032390,0.031950,0.033712,0.029075,0.029509,0.031462,0.041333
697,Wavelength_897,0.041000,0.053464,0.138919,0.040244,0.051583,0.044412,0.040553,0.047978,0.030402,...,0.031025,0.031465,0.033005,0.032390,0.031950,0.033933,0.029292,0.029726,0.031679,0.041333
698,Wavelength_898,0.041000,0.053464,0.138919,0.040244,0.051583,0.044720,0.040863,0.048339,0.030402,...,0.031245,0.031685,0.033005,0.032390,0.032170,0.033933,0.029509,0.029509,0.031679,0.041667
699,Wavelength_899,0.041000,0.054319,0.138919,0.040244,0.051983,0.044720,0.040553,0.048339,0.030637,...,0.031025,0.031465,0.033005,0.032390,0.031950,0.033712,0.029292,0.029509,0.031462,0.041667


In [82]:
# Extract the two columns into a new DataFrame
new_df = corrected_data[['B_B', 'C7_buffer']].copy()
new_df

,B_B,C7_buffer
0,2.646333,2.677333
1,2.616000,2.643333
2,2.643667,2.674000
3,2.696000,2.733333
4,2.738667,2.777333
...,...,...
696,0.041000,0.041333
697,0.041000,0.041333
698,0.041000,0.041667
699,0.041000,0.041667


In [79]:
import pandas as pd
import re

# Create new DataFrame to store normalized values
normalized_df = pd.DataFrame()
normalized_df["Wavelength"] = corrected_data["Wavelength"]

# Store summary info
normalization_summary = []

# Loop through all columns
for col in corrected_data.columns:
    # Match columns like C7_PN1_1, C7_PB1_1, C7_PC1_1
    match = re.match(r'C7_P([NBC])(\d+)_(\d+)', col)
    if match:
        letter = match.group(1)   # N, B, or C
        pc = match.group(2)       # Number
        rep = match.group(3)      # Replicate

        # Use the same letter as the original column for baseline
        b_col = f'B_P{letter}{pc}'
        norm_col = f'Norm_C7_P{letter}{pc}_{rep}'

        # Perform normalization
        normalized_df[norm_col] = corrected_data[col] / corrected_data[b_col]

        # Record in summary
        normalization_summary.append({
            "Normalized Column": norm_col,
            "Original Column": col,
            "Baseline Column": b_col
        })


In [80]:
normalized_df

,Wavelength,Norm_C7_PB10_1,Norm_C7_PB10_2,Norm_C7_PB10_3,Norm_C7_PB11_1,Norm_C7_PB11_2,Norm_C7_PB11_3,Norm_C7_PB12_1,Norm_C7_PB12_2,Norm_C7_PB12_3,...,Norm_C7_PN6_3,Norm_C7_PN7_1,Norm_C7_PN7_2,Norm_C7_PN7_3,Norm_C7_PN8_1,Norm_C7_PN8_2,Norm_C7_PN8_3,Norm_C7_PN9_1,Norm_C7_PN9_2,Norm_C7_PN9_3
0,Wavelength_200,1.000125,0.998749,1.002753,0.999750,1.001376,1.001376,1.002130,1.002130,1.003884,...,1.002601,1.001239,1.000000,1.002726,1.001361,1.003216,1.003093,1.001739,1.000248,1.002608
1,Wavelength_201,0.998985,0.998985,1.000888,1.001524,1.001651,1.001143,0.998604,0.998731,0.998731,...,1.000627,1.000754,0.998618,1.003392,1.000626,1.002004,1.002129,1.000754,1.001509,1.002389
2,Wavelength_202,0.999247,0.997867,1.000251,1.001511,1.003904,1.002897,1.000880,1.000629,1.000503,...,1.003107,0.999628,0.999379,1.001862,0.999009,1.000372,1.003345,1.000373,1.000995,1.003607
3,Wavelength_203,0.997172,0.998401,0.999754,1.000985,1.002585,1.000985,0.999140,0.999140,1.000737,...,1.001704,0.999514,0.999271,1.002553,0.998666,1.000000,1.000364,0.999757,1.000000,0.999757
4,Wavelength_204,0.997946,0.996616,0.998671,1.002791,1.002670,1.002549,1.001818,1.002181,1.000727,...,1.002878,1.000838,0.999401,1.002155,0.999881,1.000119,1.001313,1.001440,0.999760,1.001680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Wavelength_896,0.984962,0.969925,1.075188,1.038462,1.100000,1.038462,1.053846,1.030769,1.084615,...,1.120805,1.014388,1.021583,1.079137,1.027972,1.013986,1.069930,0.985294,1.000000,1.066176
697,Wavelength_897,0.984962,0.962406,1.067669,1.038462,1.100000,1.030769,1.062016,1.046512,1.093023,...,1.113333,1.021739,1.036232,1.086957,1.020833,1.006944,1.069444,1.007463,1.022388,1.089552
698,Wavelength_898,0.984962,0.962406,1.052632,1.038462,1.100000,1.030769,1.062016,1.054264,1.093023,...,1.120805,1.028986,1.043478,1.086957,1.020833,1.013889,1.069444,1.014925,1.014925,1.089552
699,Wavelength_899,0.984962,0.969925,1.067669,1.038462,1.100000,1.046154,1.053846,1.046154,1.084615,...,1.120805,1.014388,1.028777,1.079137,1.035211,1.021127,1.077465,1.007463,1.014925,1.082090


In [61]:
normalization_summary

[{'Normalized Column': 'Norm_C7_PB10_1',
  'Original Column': 'C7_PB10_1',
  'Baseline Column': 'B_PB10'},
 {'Normalized Column': 'Norm_C7_PB10_2',
  'Original Column': 'C7_PB10_2',
  'Baseline Column': 'B_PB10'},
 {'Normalized Column': 'Norm_C7_PB10_3',
  'Original Column': 'C7_PB10_3',
  'Baseline Column': 'B_PB10'},
 {'Normalized Column': 'Norm_C7_PB11_1',
  'Original Column': 'C7_PB11_1',
  'Baseline Column': 'B_PB11'},
 {'Normalized Column': 'Norm_C7_PB11_2',
  'Original Column': 'C7_PB11_2',
  'Baseline Column': 'B_PB11'},
 {'Normalized Column': 'Norm_C7_PB11_3',
  'Original Column': 'C7_PB11_3',
  'Baseline Column': 'B_PB11'},
 {'Normalized Column': 'Norm_C7_PB12_1',
  'Original Column': 'C7_PB12_1',
  'Baseline Column': 'B_PB12'},
 {'Normalized Column': 'Norm_C7_PB12_2',
  'Original Column': 'C7_PB12_2',
  'Baseline Column': 'B_PB12'},
 {'Normalized Column': 'Norm_C7_PB12_3',
  'Original Column': 'C7_PB12_3',
  'Baseline Column': 'B_PB12'},
 {'Normalized Column': 'Norm_C7_PB13_

In [84]:
# Subtract BB from each row of each column in normalized_df, excluding "Time"
for col in normalized_df.columns:
    if col != "Wavelength":
        normalized_df[col] = normalized_df[col] / new_df["B_B"]


In [85]:
normalized_df

,Wavelength,Norm_C7_PB10_1,Norm_C7_PB10_2,Norm_C7_PB10_3,Norm_C7_PB11_1,Norm_C7_PB11_2,Norm_C7_PB11_3,Norm_C7_PB12_1,Norm_C7_PB12_2,Norm_C7_PB12_3,...,Norm_C7_PN6_3,Norm_C7_PN7_1,Norm_C7_PN7_2,Norm_C7_PN7_3,Norm_C7_PN8_1,Norm_C7_PN8_2,Norm_C7_PN8_3,Norm_C7_PN9_1,Norm_C7_PN9_2,Norm_C7_PN9_3
0,Wavelength_200,0.377929,0.377409,0.378922,0.377787,0.378401,0.378401,0.378686,0.378686,0.379349,...,0.378864,0.378350,0.377881,0.378912,0.378396,0.379097,0.379050,0.378538,0.377975,0.378867
1,Wavelength_201,0.381875,0.381875,0.382603,0.382845,0.382894,0.382700,0.381729,0.381778,0.381778,...,0.382503,0.382551,0.381735,0.383559,0.382502,0.383029,0.383077,0.382551,0.382840,0.383176
2,Wavelength_202,0.377978,0.377456,0.378357,0.378834,0.379739,0.379358,0.378596,0.378500,0.378453,...,0.379438,0.378122,0.378028,0.378967,0.377888,0.378403,0.379528,0.378404,0.378639,0.379627
3,Wavelength_203,0.369871,0.370327,0.370829,0.371285,0.371879,0.371285,0.370601,0.370601,0.371193,...,0.371552,0.370740,0.370649,0.371867,0.370425,0.370920,0.371055,0.370830,0.370920,0.370830
4,Wavelength_204,0.364391,0.363906,0.364656,0.366160,0.366116,0.366072,0.365805,0.365938,0.365407,...,0.366192,0.365447,0.364923,0.365928,0.365098,0.365185,0.365621,0.365667,0.365054,0.365755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Wavelength_896,24.023473,23.656703,26.224097,25.328330,26.829268,25.328330,25.703565,25.140713,26.454034,...,27.336716,24.741183,24.916652,26.320407,25.072488,24.731366,26.095855,24.031564,24.390244,26.004304
697,Wavelength_897,24.023473,23.473317,26.040712,25.328330,26.829268,25.140713,25.902817,25.524674,26.659104,...,27.154472,24.920467,25.273948,26.511135,24.898374,24.559621,26.084011,24.572261,24.936294,26.574445
698,Wavelength_898,24.023473,23.473317,25.673941,25.328330,26.829268,25.140713,25.902817,25.713746,26.659104,...,27.336716,25.097207,25.450689,26.511135,24.898374,24.728997,26.084011,24.754277,24.754277,26.574445
699,Wavelength_899,24.023473,23.656703,26.040712,25.328330,26.829268,25.515947,25.703565,25.515947,26.454034,...,27.336716,24.741183,25.092121,26.320407,25.249055,24.905531,26.279629,24.572261,24.754277,26.392428


In [86]:
# Subtract BB from each row of each column in normalized_df, excluding "Time"
for col in normalized_df.columns:
    if col != "Wavelength":
        normalized_df[col] = normalized_df[col] / new_df["C7_buffer"]


In [96]:
normalized_df

,Wavelength,Norm_C7_PB1_1,Norm_C7_PC1_1,Norm_C7_PN1_1,Norm_C7_PB1_2,Norm_C7_PC1_2,Norm_C7_PN1_2,Norm_C7_PB1_3,Norm_C7_PC1_3,Norm_C7_PN1_3,...,Norm_C7_PC75_3,Norm_C7_PC76_1,Norm_C7_PC76_2,Norm_C7_PC76_3,Norm_C7_PC77_1,Norm_C7_PC77_2,Norm_C7_PC77_3,Norm_C7_PC78_1,Norm_C7_PC78_2,Norm_C7_PC78_3
0,Wavelength_200,0.141442,0.140426,0.141280,0.141602,0.140670,0.141193,0.141637,0.140932,0.141297,...,0.141653,0.141387,0.141669,0.141774,0.141141,0.140807,0.140965,0.141317,0.141000,0.140921
1,Wavelength_201,0.144798,0.144233,0.145047,0.144798,0.144814,0.144957,0.144761,0.144523,0.144416,...,0.145183,0.144924,0.144979,0.144997,0.144614,0.144486,0.144760,0.144669,0.144724,0.144815
2,Wavelength_202,0.142101,0.140914,0.141651,0.141905,0.141354,0.141302,0.141994,0.141407,0.141180,...,0.141797,0.141459,0.142060,0.141689,0.141708,0.141211,0.141389,0.141584,0.141601,0.141858
3,Wavelength_203,0.136003,0.135174,0.135834,0.135686,0.135521,0.135686,0.136053,0.135603,0.135604,...,0.135986,0.135503,0.135901,0.135553,0.135786,0.135786,0.136052,0.135886,0.136153,0.135969
4,Wavelength_204,0.131615,0.131063,0.131269,0.131759,0.131015,0.131597,0.131647,0.131189,0.131253,...,0.132077,0.131093,0.131661,0.131172,0.131361,0.131044,0.131393,0.131567,0.131647,0.131408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Wavelength_896,608.819452,608.969315,598.007171,608.819452,604.248623,613.848420,627.552358,613.690008,613.848420,...,612.104701,602.553163,627.486398,615.019780,615.375969,560.582219,594.301450,618.185905,600.623806,584.817916
697,Wavelength_897,613.690008,608.969315,597.954367,618.410700,604.248623,609.756098,632.572777,613.690008,609.756098,...,603.199580,602.553163,627.486398,615.019780,611.011601,556.606458,590.086546,625.633928,604.305499,591.863915
698,Wavelength_898,608.780488,594.657375,593.223113,608.780488,594.657375,605.009003,627.512195,603.948897,608.937633,...,585.365854,593.552789,618.113594,605.833191,606.123508,556.305138,585.365854,617.294900,599.556541,580.044346
699,Wavelength_899,599.193394,594.731707,593.170732,599.193394,604.097561,604.878049,617.630113,608.780488,612.682927,...,589.607635,593.552789,618.113594,605.833191,606.123508,556.305138,589.517385,613.240418,592.334495,580.139373


In [99]:
import re

# Keep Wavelength first
wavelength_col = ['Wavelength']

# Get all other columns
other_cols = [c for c in normalized_df.columns if c != 'Wavelength']

# Initialize group dictionary
grouped_cols = {'PB': [], 'PN': [], 'PC': [], 'other': []}

# Loop through columns safely
for col in other_cols:
    m = re.match(r'.*_(P[BPNC]\d+)_\d+', col)  # match PB, PN, PC pattern
    if m:
        # Extract PB / PN / PC and number
        letter_num = m.group(1)  # e.g., PB10
        letter = letter_num[:2]  # PB / PN / PC
        num = int(letter_num[2:])  # 10
        if letter in grouped_cols:
            grouped_cols[letter].append((num, col))
        else:
            grouped_cols['other'].append(col)
    else:
        grouped_cols['other'].append(col)

# Flatten grouped columns
sorted_cols = wavelength_col
for t in ['PB', 'PN', 'PC']:
    sorted_cols.extend([col for num, col in sorted(grouped_cols[t], key=lambda x: x[0])])

# Add other columns
sorted_cols.extend(grouped_cols['other'])

# Reorder DataFrame
normalized_df = normalized_df[sorted_cols]


In [100]:
normalized_df

,Wavelength,Norm_C7_PB1_1,Norm_C7_PB1_2,Norm_C7_PB1_3,Norm_C7_PB2_1,Norm_C7_PB2_2,Norm_C7_PB2_3,Norm_C7_PB3_1,Norm_C7_PB3_2,Norm_C7_PB3_3,...,Norm_C7_PC75_3,Norm_C7_PC76_1,Norm_C7_PC76_2,Norm_C7_PC76_3,Norm_C7_PC77_1,Norm_C7_PC77_2,Norm_C7_PC77_3,Norm_C7_PC78_1,Norm_C7_PC78_2,Norm_C7_PC78_3
0,Wavelength_200,0.141442,0.141602,0.141637,0.141194,0.141229,0.141335,0.141229,0.141388,0.141635,...,0.141653,0.141387,0.141669,0.141774,0.141141,0.140807,0.140965,0.141317,0.141000,0.140921
1,Wavelength_201,0.144798,0.144798,0.144761,0.144742,0.144412,0.144889,0.144651,0.144632,0.144724,...,0.145183,0.144924,0.144979,0.144997,0.144614,0.144486,0.144760,0.144669,0.144724,0.144815
2,Wavelength_202,0.142101,0.141905,0.141994,0.141566,0.141335,0.141495,0.141779,0.141761,0.141956,...,0.141797,0.141459,0.142060,0.141689,0.141708,0.141211,0.141389,0.141584,0.141601,0.141858
3,Wavelength_203,0.136003,0.135686,0.136053,0.135552,0.135836,0.135852,0.135869,0.136002,0.136169,...,0.135986,0.135503,0.135901,0.135553,0.135786,0.135786,0.136052,0.135886,0.136153,0.135969
4,Wavelength_204,0.131615,0.131759,0.131647,0.131440,0.131663,0.131440,0.131456,0.131440,0.131964,...,0.132077,0.131093,0.131661,0.131172,0.131361,0.131044,0.131393,0.131567,0.131647,0.131408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Wavelength_896,608.819452,608.819452,627.552358,626.966955,613.136802,617.746853,594.271557,602.641579,636.121667,...,612.104701,602.553163,627.486398,615.019780,615.375969,560.582219,594.301450,618.185905,600.623806,584.817916
697,Wavelength_897,613.690008,618.410700,632.572777,626.966955,608.526751,617.746853,594.271557,602.641579,627.751645,...,603.199580,602.553163,627.486398,615.019780,611.011601,556.606458,590.086546,625.633928,604.305499,591.863915
698,Wavelength_898,608.780488,608.780488,627.512195,621.951220,603.658537,612.804878,593.728223,602.090592,635.540070,...,585.365854,593.552789,618.113594,605.833191,606.123508,556.305138,585.365854,617.294900,599.556541,580.044346
699,Wavelength_899,599.193394,599.193394,617.630113,621.951220,603.658537,612.804878,589.517385,597.820446,626.881162,...,589.607635,593.552789,618.113594,605.833191,606.123508,556.305138,589.517385,613.240418,592.334495,580.139373


In [101]:
normalized_df.to_csv("Preprocessed_spectra_all.csv", index = False)

# Batch correction 

In [5]:
AA = pd.read_csv("preprocessed_spectra_all.csv")
AA

,Wavelength,Norm_C7_PB10_1,Norm_C7_PB10_2,Norm_C7_PB10_3,Norm_C7_PB11_1,Norm_C7_PB11_2,Norm_C7_PB11_3,Norm_C7_PB12_1,Norm_C7_PB12_2,Norm_C7_PB12_3,...,Norm_C7_PN6_3,Norm_C7_PN7_1,Norm_C7_PN7_2,Norm_C7_PN7_3,Norm_C7_PN8_1,Norm_C7_PN8_2,Norm_C7_PN8_3,Norm_C7_PN9_1,Norm_C7_PN9_2,Norm_C7_PN9_3
0,Wavelength_200,0.141159,0.140964,0.141529,0.141106,0.141335,0.141335,0.141442,0.141442,0.141689,...,0.141508,0.141316,0.141141,0.141526,0.141333,0.141595,0.141577,0.141386,0.141176,0.141509
1,Wavelength_201,0.144467,0.144467,0.144742,0.144834,0.144853,0.144779,0.144412,0.144430,0.144430,...,0.144705,0.144723,0.144414,0.145104,0.144705,0.144904,0.144922,0.144723,0.144832,0.144959
2,Wavelength_202,0.141353,0.141158,0.141495,0.141673,0.142012,0.141869,0.141584,0.141548,0.141531,...,0.141899,0.141407,0.141372,0.141723,0.141319,0.141512,0.141933,0.141512,0.141600,0.141970
3,Wavelength_203,0.135319,0.135485,0.135669,0.135836,0.136053,0.135836,0.135586,0.135586,0.135802,...,0.135934,0.135636,0.135603,0.136049,0.135521,0.135702,0.135752,0.135669,0.135702,0.135669
4,Wavelength_204,0.131202,0.131027,0.131297,0.131839,0.131823,0.131807,0.131711,0.131759,0.131567,...,0.131850,0.131582,0.131393,0.131755,0.131456,0.131488,0.131645,0.131661,0.131440,0.131693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Wavelength_896,581.213064,572.339582,634.453956,612.782182,649.095201,612.782182,621.860437,608.243055,640.016946,...,661.372169,598.577000,602.822227,636.784042,606.592463,598.339505,631.351339,581.408803,590.086546,629.136391
697,Wavelength_897,581.213064,567.902841,630.017215,612.782182,649.095201,608.243055,626.681061,617.532432,644.978318,...,656.963021,602.914514,611.466493,641.398420,602.380016,594.184369,631.064778,594.490177,603.297439,642.930117
698,Wavelength_898,576.563360,563.359619,616.174583,607.879925,643.902439,603.377111,621.667612,617.129892,639.818491,...,656.081192,602.332980,610.816543,636.267232,597.560976,593.495935,626.016260,594.102657,594.102657,637.786676
699,Wavelength_899,576.563360,567.760866,624.977077,607.879925,643.902439,612.382739,616.885553,612.382739,634.896811,...,656.081192,593.788384,602.210914,631.689770,605.977327,597.732738,630.711096,589.734256,594.102657,633.418274


In [3]:
METADATA = pd.read_csv("Spectral_All_Plates_SampleID_and_Batch.csv")
METADATA

,Batch,Sample ID
0,P_1,B_PC1
1,P_1,C7_PC2_3
2,P_1,B_PC2
3,P_1,C7_PC1_1
4,P_1,C7_PC2_1
...,...,...
693,P_38,C7_PB49_3
694,P_38,B_PB50
695,P_38,C7_PB50_1
696,P_38,C7_PB50_2


In [7]:
import pandas as pd
import numpy as np
from pathlib import Path
from combat.pycombat import pycombat

# ===========================================================
# CONFIGURATION
# ===========================================================
RAW_DATA_FILE = Path("preprocessed_spectra_all.csv")
METADATA_FILE = Path("Spectral_All_Plates_SampleID_and_Batch.csv")
FINAL_CORRECTED_FILE = RAW_DATA_FILE.with_name("spectra_batch_corrected_only.csv")

print("=== Running Batch Correction Only (pyComBat) ===")

# ===========================================================
# 1️⃣ Load input files
# ===========================================================
print(f"Loading normalized spectra: {RAW_DATA_FILE}")
data_df = pd.read_csv(RAW_DATA_FILE, low_memory=False)

print(f"Loading metadata: {METADATA_FILE}")
meta_df = pd.read_csv(METADATA_FILE)

# ===========================================================
# 2️⃣ Clean up sample names
# ===========================================================
# Remove 'Norm_' prefix from spectra columns
data_df.columns = data_df.columns.str.replace(r"^Norm_", "", regex=True)

# Remove 'C7_' prefix from spectra columns if present
data_df.columns = data_df.columns.str.replace(r"^C7_", "", regex=True)

# Clean metadata Sample IDs similarly
meta_df["Sample ID"] = meta_df["Sample ID"].str.replace(r"^C7_", "", regex=True)

# ===========================================================
# 3️⃣ Prepare matrices
# ===========================================================
# Set first column (Wavelength) as index
data_df.set_index(data_df.columns[0], inplace=True)

# Ensure metadata has Sample ID as index
meta_df.set_index("Sample ID", inplace=True)

# ===========================================================
# 4️⃣ Match samples between data and metadata
# ===========================================================
common_samples = sorted(set(data_df.columns) & set(meta_df.index))

if not common_samples:
    print("\n🔍 Debug Info:")
    print(f"Example spectra column names: {list(data_df.columns[:5])}")
    print(f"Example metadata Sample IDs: {list(meta_df.index[:5])}")
    raise ValueError("❌ No overlapping Sample IDs between spectra and metadata.")

print(f"✅ Found {len(common_samples)} common samples.")

# Subset to common samples
data_filtered = data_df[common_samples]
batch_info = meta_df.loc[common_samples, "Batch"]

# ===========================================================
# 5️⃣ Handle missing values
# ===========================================================
if data_filtered.isnull().values.any():
    print("⚠️ NaN values found — imputing with row mean...")
    data_filtered = data_filtered.apply(lambda row: row.fillna(row.mean()), axis=1)
else:
    print("✅ No NaN values found.")

# ===========================================================
# 6️⃣ Run ComBat
# ===========================================================
print("🚀 Running pyComBat batch correction...")
corrected_data = pycombat(data_filtered, batch_info)
print("✅ Batch correction complete.")

# ===========================================================
# 7️⃣ Save result
# ===========================================================
corrected_data.index.name = data_df.index.name
corrected_data.reset_index().to_csv(FINAL_CORRECTED_FILE, index=False)

print(f"\n✅ Saved batch-corrected data → {FINAL_CORRECTED_FILE}")


=== Running Batch Correction Only (pyComBat) ===
Loading normalized spectra: preprocessed_spectra_all.csv
Loading metadata: Spectral_All_Plates_SampleID_and_Batch.csv
✅ Found 522 common samples.
✅ No NaN values found.
🚀 Running pyComBat batch correction...
Found 38 batches.
Adjusting for 0 covariate(s) or covariate level(s).
Standardizing Data across genes.
Fitting L/S model and finding priors.
Finding parametric adjustments.
Adjusting the Data
✅ Batch correction complete.

✅ Saved batch-corrected data → spectra_batch_corrected_only.csv


In [8]:
oo = pd.read_csv("spectra_batch_corrected_only.csv")
oo

,Wavelength,PB10_1,PB10_2,PB10_3,PB11_1,PB11_2,PB11_3,PB12_1,PB12_2,PB12_3,...,PN6_3,PN7_1,PN7_2,PN7_3,PN8_1,PN8_2,PN8_3,PN9_1,PN9_2,PN9_3
0,Wavelength_200,0.141098,0.140901,0.141474,0.141044,0.141277,0.141277,0.141385,0.141385,0.141636,...,0.141412,0.141216,0.141038,0.141431,0.141234,0.141501,0.141483,0.141288,0.141073,0.141414
1,Wavelength_201,0.144471,0.144471,0.144736,0.144824,0.144842,0.144771,0.144418,0.144436,0.144436,...,0.144615,0.144635,0.144310,0.145035,0.144615,0.144824,0.144843,0.144635,0.144749,0.144883
2,Wavelength_202,0.141330,0.141137,0.141470,0.141646,0.141980,0.141839,0.141558,0.141523,0.141505,...,0.141836,0.141324,0.141287,0.141653,0.141233,0.141434,0.141871,0.141434,0.141525,0.141910
3,Wavelength_203,0.135345,0.135499,0.135669,0.135824,0.136026,0.135824,0.135592,0.135592,0.135793,...,0.135889,0.135589,0.135556,0.136005,0.135473,0.135656,0.135706,0.135622,0.135656,0.135622
4,Wavelength_204,0.131199,0.131035,0.131289,0.131800,0.131785,0.131770,0.131680,0.131725,0.131544,...,0.131786,0.131508,0.131313,0.131687,0.131378,0.131410,0.131573,0.131590,0.131361,0.131623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Wavelength_896,577.569272,568.891643,629.635050,608.441608,643.953138,608.441608,617.319491,604.002667,635.075256,...,650.907688,590.259075,594.359185,627.160069,598.000542,590.029698,621.913074,573.677746,582.058854,619.773840
697,Wavelength_897,577.269382,564.213785,625.139905,608.234581,643.852927,603.782287,621.867572,612.893957,639.814801,...,646.816339,594.033446,602.385169,631.616202,593.511463,585.507728,621.524536,585.806375,594.407404,633.112033
698,Wavelength_898,573.040831,560.218851,611.506773,603.451939,638.432933,599.079315,616.840982,612.434462,634.467064,...,645.357008,593.193574,601.426997,626.127267,588.562273,584.617091,616.178547,585.205924,585.205924,627.601910
699,Wavelength_899,572.549988,564.003293,619.556815,602.956502,637.932210,607.328465,611.700429,607.328465,629.188283,...,645.619932,584.900155,593.109997,621.844448,596.781300,588.744905,620.890487,580.948402,585.206492,623.529303


In [11]:

# 2️⃣ Use the first column ('Wavelength') as index before transpose
oo_t = oo.set_index("Wavelength").T

# 3️⃣ Reset index so sample names become a column again
oo_t.reset_index(inplace=True)

# 4️⃣ Rename for clarity
oo_t.rename(columns={"index": "Sample"}, inplace=True)

oo_t


Wavelength,Sample,Wavelength_200,Wavelength_201,Wavelength_202,Wavelength_203,Wavelength_204,Wavelength_205,Wavelength_206,Wavelength_207,Wavelength_208,...,Wavelength_891,Wavelength_892,Wavelength_893,Wavelength_894,Wavelength_895,Wavelength_896,Wavelength_897,Wavelength_898,Wavelength_899,Wavelength_900
0,PB10_1,0.141098,0.144471,0.141330,0.135345,0.131199,0.127653,0.125949,0.126938,0.130218,...,581.762387,568.831226,569.450981,577.718064,576.664430,577.569272,577.269382,573.040831,572.549988,564.862391
1,PB10_2,0.140901,0.144471,0.141137,0.135499,0.131035,0.127418,0.126025,0.126697,0.130311,...,564.380747,560.306223,560.835516,564.659867,567.866504,568.891643,564.213785,560.218851,564.003293,556.612305
2,PB10_3,0.141474,0.144736,0.141470,0.135669,0.131289,0.127640,0.126314,0.127089,0.130545,...,625.216487,615.718743,612.528308,616.892653,620.654057,629.635050,625.139905,611.506773,619.556815,606.112822
3,PB11_1,0.141044,0.144824,0.141646,0.135824,0.131800,0.127931,0.126101,0.126606,0.130514,...,603.756847,599.193352,600.135293,608.689427,607.964742,608.441608,608.234581,603.451939,602.956502,602.749325
4,PB11_2,0.141277,0.144842,0.141980,0.136026,0.131785,0.128001,0.126177,0.127074,0.130747,...,643.467209,638.736252,640.097798,644.314866,643.968560,643.953138,643.852927,638.432933,637.932210,636.765065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,PN8_2,0.141501,0.144824,0.141434,0.135656,0.131410,0.127983,0.126253,0.126950,0.130161,...,589.218107,588.546015,584.369083,585.237789,589.024745,590.029698,585.507728,584.617091,588.744905,579.901073
518,PN8_3,0.141483,0.144843,0.141871,0.135706,0.131573,0.127983,0.126238,0.127077,0.130396,...,630.176805,628.713882,628.292621,621.503597,629.235370,621.913074,621.524536,616.178547,620.890487,611.751776
519,PN9_1,0.141288,0.144635,0.141434,0.135622,0.131590,0.127872,0.126180,0.127046,0.130381,...,581.026367,576.495655,576.382985,581.208255,585.273753,573.677746,585.806375,585.205924,580.948402,571.938397
520,PN9_2,0.141073,0.144749,0.141525,0.135656,0.131361,0.127712,0.125903,0.126855,0.130145,...,589.707166,580.752250,584.845865,589.808604,589.564887,582.058854,594.407404,585.205924,585.206492,580.372935


In [14]:
oo_t.columns

Index(['Sample', 'Wavelength_200', 'Wavelength_201', 'Wavelength_202',
       'Wavelength_203', 'Wavelength_204', 'Wavelength_205', 'Wavelength_206',
       'Wavelength_207', 'Wavelength_208',
       ...
       'Wavelength_891', 'Wavelength_892', 'Wavelength_893', 'Wavelength_894',
       'Wavelength_895', 'Wavelength_896', 'Wavelength_897', 'Wavelength_898',
       'Wavelength_899', 'Wavelength_900'],
      dtype='object', name='Wavelength', length=702)

In [9]:
import pandas as pd
NN = pd.read_csv("Spectra_after_Normalisation.csv")
NN

,Sample,Metadata,Wavelength_200,Wavelength_201,Wavelength_202,Wavelength_203,Wavelength_204,Wavelength_205,Wavelength_206,Wavelength_207,...,Wavelength_892,Wavelength_893,Wavelength_894,Wavelength_895,Wavelength_896,Wavelength_897,Wavelength_898,Wavelength_899,Wavelength_900,mean_abs
0,PC1_1,Cancer,0.140426,0.144233,0.140914,0.135174,0.131063,0.127541,0.125622,0.126560,...,604.058268,604.058268,608.969311,608.969311,608.969311,608.969311,594.657380,594.731698,589.789184,537.393022
1,PC1_2,Cancer,0.140670,0.144814,0.141354,0.135521,0.131015,0.127587,0.125785,0.126979,...,599.375650,594.693019,608.969311,608.969311,604.248623,604.248623,594.657380,604.097556,589.789184,539.041687
2,PC1_3,Cancer,0.140932,0.144523,0.141407,0.135603,0.131189,0.127662,0.125858,0.127293,...,608.740885,608.740885,608.969311,613.689998,613.689998,613.689998,603.948892,608.780478,598.933207,551.939203
3,PC2_1,Cancer,0.141730,0.145064,0.142036,0.136326,0.132161,0.128173,0.126432,0.127497,...,674.752858,674.752858,680.238653,680.238653,680.238653,672.043015,662.068974,670.731707,664.738599,609.715804
4,PC2_2,Cancer,0.140916,0.144362,0.141320,0.135538,0.131174,0.127618,0.125728,0.126875,...,536.550468,540.615241,540.912667,540.912667,545.010486,545.010486,536.921783,540.650402,532.591768,473.007038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,PB49_2,Benign,0.141370,0.144834,0.141886,0.135786,0.131790,0.128134,0.126364,0.127085,...,617.089800,617.089800,608.243100,617.321300,617.532400,612.608900,607.879900,607.708100,602.807200,554.988190
518,PB49_3,Benign,0.141459,0.145110,0.141726,0.136053,0.131854,0.128195,0.126648,0.127160,...,648.851700,653.389200,649.095200,649.095200,654.127000,648.644700,643.902400,638.987200,638.266400,592.778661
519,PB50_1,Benign,0.141353,0.144614,0.141389,0.135502,0.131107,0.127585,0.125857,0.126663,...,572.603300,576.844800,577.258600,569.012000,577.258600,581.534600,576.882300,572.640500,568.113200,516.957789
520,PB50_2,Benign,0.141547,0.144742,0.141371,0.135652,0.131091,0.127600,0.126006,0.126874,...,568.361800,576.844800,577.258600,569.012000,581.534600,585.810600,576.882300,568.398700,572.290600,514.388671


In [15]:
# Keep only the desired columns
NN = NN[["Sample", "Metadata"]]
NN

,Sample,Metadata
0,PC1_1,Cancer
1,PC1_2,Cancer
2,PC1_3,Cancer
3,PC2_1,Cancer
4,PC2_2,Cancer
...,...,...
517,PB49_2,Benign
518,PB49_3,Benign
519,PB50_1,Benign
520,PB50_2,Benign


In [16]:
merged_df = pd.merge(NN, oo_t, on="Sample", how="left")
merged_df

,Sample,Metadata,Wavelength_200,Wavelength_201,Wavelength_202,Wavelength_203,Wavelength_204,Wavelength_205,Wavelength_206,Wavelength_207,...,Wavelength_891,Wavelength_892,Wavelength_893,Wavelength_894,Wavelength_895,Wavelength_896,Wavelength_897,Wavelength_898,Wavelength_899,Wavelength_900
0,PC1_1,Cancer,0.140882,0.144546,0.141296,0.135543,0.131375,0.127777,0.125928,0.126840,...,623.200979,618.258041,618.345611,623.218749,622.999774,622.978034,623.061955,612.258285,612.135381,607.303469
1,PC1_2,Cancer,0.141040,0.144935,0.141594,0.135771,0.131344,0.127807,0.126039,0.127114,...,617.256948,615.333707,612.452274,623.218749,622.999774,620.026185,620.093245,612.258285,618.024574,607.303469
2,PC1_3,Cancer,0.141210,0.144740,0.141630,0.135826,0.131458,0.127856,0.126089,0.127319,...,626.172995,621.182375,621.292280,623.218749,625.952330,625.929883,626.030664,618.215681,620.969170,613.090751
3,PC2_1,Cancer,0.141729,0.145104,0.142056,0.136300,0.132098,0.128193,0.126479,0.127452,...,668.467063,662.407356,662.832120,667.944093,667.575160,667.542749,662.727213,655.480216,659.923730,654.739226
4,PC2_2,Cancer,0.141200,0.144632,0.141571,0.135783,0.131448,0.127827,0.126000,0.127046,...,585.333759,576.098898,578.422346,580.509543,580.433767,582.984585,582.840063,575.240263,578.129379,571.103022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,PB49_2,Benign,0.141322,0.144828,0.141907,0.135784,0.131806,0.128131,0.126366,0.127092,...,621.948533,616.652426,615.990556,607.515903,617.935889,616.591255,612.147017,607.509472,608.310652,602.149593
518,PB49_3,Benign,0.141423,0.145137,0.141731,0.136083,0.131869,0.128195,0.126648,0.127174,...,651.775551,650.983249,655.642715,652.466829,651.772665,656.557920,652.420535,647.462646,642.746012,640.905705
519,PB50_1,Benign,0.141302,0.144581,0.141360,0.135467,0.131122,0.127551,0.125861,0.126633,...,573.209457,568.567951,572.028379,573.422689,566.490179,572.606203,577.418405,573.129477,569.704549,564.229945
520,PB50_2,Benign,0.141524,0.144725,0.141340,0.135635,0.131106,0.127567,0.126009,0.126863,...,568.562494,563.983400,572.028379,573.422689,566.490179,577.276221,582.197237,573.129477,565.034743,568.795638


In [17]:
merged_df.to_csv("Final_Spectra_data.csv" , index = False )